In [1]:
# prepare the R environment
if (!require("pacman")) install.packages("pacman")
pacman::p_load(
  dplyr,         # Data munging functions
  zoo,              # Feature engineering rolling aggregates
  data.table,       # Feature engineering
  ggplot2,          # Graphics
  scales,           # Time formatted axis
  readr,            # Reading input files
  stringr,          # String functions
  reshape2,         # restructure and aggregate data 
  randomForest,     # Random forests
  corrplot,         # correlation plots
  Metrics,          # Eval metrics for ML
  vcd               # Visualizing discrete distributions
)

# set options for plots
options(repr.plot.width=6, repr.plot.height=6)

Loading required package: pacman
Warning message:
"package 'pacman' was built under R version 3.5.1"Installing package into 'C:/Program Files/R/R-3.5.1/library'
(as 'lib' is unspecified)
Warning message in utils::install.packages(package, ...):
"'lib = "C:/Program Files/R/R-3.5.1/library"' is not writable"

ERROR: Error in utils::install.packages(package, ...): unable to install packages


In [3]:
# Load the googs data
googFile <-"C:\\Users\\Mirela\\RStudioProjects\\Regression_file\\Regression_Data\\goog.csv"
nasdaqFile<-"C:\\Users\\Mirela\\RStudioProjects\\Regression_file\\Regression_Data\\Nasdaq.csv"
tbondsFile <-"C:\\Users\\Mirela\\RStudioProjects\\Regression_file\\Regression_Data\\tbond5yr.csv"
if(!file.exists(googFile)){tryCatch(googFile)}
if(file.exists(googFile)) goog_original <- read.csv(googFile)
if(!file.exists(nasdaqFile)){tryCatch(nasdaqFile)}
if(file.exists(nasdaqFile)) nasdaqFile_original <- read.csv(nasdaqFile)
if(!file.exists(tbondsFile)){tryCatch(tbondsFile)}
if(file.exists(tbondsFile)) tbondsFile_original <- read.csv(tbondsFile)
head(goog_original)
head(nasdaqFile_original)
head(tbondsFile_original)

Date,Open,High,Low,Close,Volume,Adj.Close
2016-02-01,750.46,757.860,743.27,752.00,10278400,752.00
2016-01-04,743.00,752.000,673.26,742.95,2632600,742.95
2015-12-01,747.11,779.980,724.17,758.88,2026100,758.88
2015-11-02,711.06,762.708,705.85,742.60,1801600,742.60
2015-10-01,608.37,730.000,599.85,710.81,2333600,710.81
2015-09-01,602.36,650.900,589.38,608.42,2398400,608.42


Date,Open,High,Low,Close,Volume,Adj.Close
2016-02-01,4587.59,4636.93,4565.37,4620.37,3966680000,4620.37
2016-01-04,4897.65,4926.73,4313.39,4613.95,2499720000,4613.95
2015-12-01,5129.64,5176.77,4871.59,5007.41,1911610000,5007.41
2015-11-02,5065.64,5163.47,4908.66,5108.67,1951903500,5108.67
2015-10-01,4624.46,5095.69,4552.34,5053.75,2016574000,5053.75
2015-09-01,4673.61,4960.87,4487.06,4620.16,2090584200,4620.16


Date,Open,High,Low,Close,Volume,Adj.Close
2016-02-01,1.343,1.383,1.334,1.383,0,1.383
2016-01-04,1.720,1.740,1.328,1.335,0,1.335
2015-12-01,1.666,1.829,1.568,1.758,0,1.758
2015-11-02,1.553,1.779,1.543,1.654,0,1.654
2015-10-01,1.365,1.558,1.215,1.528,0,1.528
2015-09-01,1.504,1.614,1.372,1.375,0,1.375
